In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.compose import ColumnTransformer

df = pd.read_csv('D:/umd/ds602/my_final_project/database/shared_overview.csv')

df1 = df.drop(['Trip','Date','Battery Temperature (Start) [°C]','Battery Temperature (End)','Battery State of Charge (Start)', 'Battery State of Charge (End)','Fan'], axis = 1)
object_cols = df1.select_dtypes(include=['object']).columns
df1 = pd.get_dummies(df1, columns=object_cols, dtype=int)

# Do the linear regression of whole overview data
X = df1.drop(['SOC'], axis=1)
y = df1['SOC']
# print(X)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    SOC   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     46.56
Date:                Sun, 13 Oct 2024   Prob (F-statistic):           3.83e-25
Time:                        16:12:04   Log-Likelihood:                 177.51
No. Observations:                  70   AIC:                            -313.0
Df Residuals:                      49   BIC:                            -265.8
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                       0.0910      0.028      3.198      0.002       0.034       0.148
Ambient Temperature (Start) [°C]           -0.0009      0.001     -0.821      0.415      -0.003       0.001
Target Cabin Temperature                   -0.0016      0.001     -1.104      0.275      -0.005       0.001
Distance [km]                               0.0086      0.001     14.257      0.000       0.007       0.010
Duration [min]                             -0.0023      0.000     -5.699      0.000      -0.003      -0.001
Mean Battery Temperature [°C]              -0.0016      0.001     -1.337      0.187      -0.004       0.001
Route/Area_FTMRoute                         0.0078      0.012      0.640      0.525      -0.017       0.032
Route/Area_FTMRoute (2x)                    0.0270      0.024      1.149      0.256      -0.020       0.074
Route/Area_FTMRoute reverse                -0.0018      0.024     -0.077      0.939      -0.049       0.046
Route/Area_Highway                          0.0425      0.018      2.324      0.024       0.006       0.079
Route/Area_Munich East                      0.0030      0.014      0.212      0.833      -0.025       0.031
Route/Area_Munich North                    -0.0152      0.009     -1.772      0.083      -0.032       0.002
Route/Area_Munich North + Fast Charging     0.0922      0.024      3.783      0.000       0.043       0.141
Route/Area_Munich Northeast                -0.0268      0.016     -1.667      0.102      -0.059       0.006
Route/Area_Munich South                    -0.0377      0.014     -2.610      0.012      -0.067      -0.009
Weather_cloudy                              0.0168      0.010      1.630      0.110      -0.004       0.037
Weather_dark                                0.0339      0.013      2.549      0.014       0.007       0.061
Weather_dark, little rainy                  0.0101      0.023      0.432      0.668      -0.037       0.057
Weather_rainy                              -0.0356      0.018     -1.997      0.051      -0.071       0.000
Weather_slightly cloudy                     0.0146      0.008      1.760      0.085      -0.002       0.031
Weather_sunny                               0.0089      0.009      0.997      0.324      -0.009       0.027
Weather_sunrise                             0.0180      0.014      1.254      0.216      -0.011       0.047
Weather_sunset                              0.0243      0.022      1.129      0.264      -0.019       0.068
==============================================================================
Omnibus:                        6.705   Durbin-Watson:                   2.290
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                6.599
Skew:                          -0.503 

In [4]:
# Use linear regression in sklearn to make the prediction
# Separate features and target variable
X = df1.drop('SOC', axis=1)
y = df1['SOC']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the scaler and fit it on the training data
# feature scaling for numerical data (except for those one-hot encoded columns)
numerical_features = [
    'Ambient Temperature (Start) [°C]',
    'Target Cabin Temperature', 
    'Distance [km]', 
    'Duration [min]',
    'Mean Battery Temperature [°C]',
]
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'  # This will leave non-numerical columns unchanged
)
# Fit on training data
X_train_scaled = preprocessor.fit_transform(X_train)

# Transform test data
X_test_scaled = preprocessor.transform(X_test)

# Fit the model on the standardized data
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict on the train and test set
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calculate MSE for training data
MSE_train = mean_squared_error(y_train, y_train_pred)
print('MSE of linear regression model on training data:', MSE_train)
# Calculate MSE for testing data
MSE_test = mean_squared_error(y_test, y_test_pred)
print('MSE of linear regression on testing model:', MSE_test)

# Calculate R-squared
R_square = r2_score(y_test, y_pred)
print('R_square of linear regression model:', R_square)

MSE of linear regression model on training data: 0.00036505293770798525
MSE of linear regression on testing model: 0.0005970968409005965
R_square of linear regression model: 0.7768121946206586


In [6]:
from sklearn.linear_model import Ridge, Lasso

# Fit Lasso and Ridge models (with intercept by default)
lasso_model = Lasso(alpha=0.01)
ridge_model = Ridge(alpha=1)

lasso_model.fit(X_train_scaled, y_train)
ridge_model.fit(X_train_scaled, y_train)

# Evaluate results
lasso_pred_train = lasso_model.predict(X_train_scaled)
ridge_pred_train = ridge_model.predict(X_train_scaled)

lasso_pred_test = lasso_model.predict(X_test_scaled)
ridge_pred_test = ridge_model.predict(X_test_scaled)

# Performance metrics
print("Lasso Regression Results:")
print(f"Intercept: {lasso_model.intercept_}")
print(f"Coefficients: {lasso_model.coef_}")
print(f"Mean Squared Error on training data: {mean_squared_error(y_train, lasso_pred_train)}")
print(f"Mean Squared Error on testing data: {mean_squared_error(y_test, lasso_pred_test)}")
print(f"R^2 Score: {r2_score(y_test, lasso_pred_test)}")

print("\nRidge Regression Results:")
print(f"Intercept: {ridge_model.intercept_}")
print(f"Coefficients: {ridge_model.coef_}")
print(f"Mean Squared Error on training data: {mean_squared_error(y_train, ridge_pred_train)}")
print(f"Mean Squared Error on testing data: {mean_squared_error(y_test, ridge_pred_test)}")
print(f"R^2 Score: {r2_score(y_test, ridge_pred_test)}")

Lasso Regression Results:
Intercept: 0.14005357142857136
Coefficients: [-0.0118153  -0.          0.07058091 -0.         -0.00220942  0.
  0.          0.          0.         -0.          0.          0.
 -0.         -0.          0.          0.          0.         -0.
  0.         -0.          0.          0.        ]
Mean Squared Error on training data: 0.0013863410803106114
Mean Squared Error on testing data: 0.00032182369154514276
R^2 Score: 0.8797060735965339

Ridge Regression Results:
Intercept: 0.1477827941855803
Coefficients: [-0.00983167 -0.0019799   0.09575213 -0.02763476 -0.01165923  0.00074703
  0.01231576  0.          0.03868517 -0.010994   -0.01739116  0.03596721
 -0.02043256 -0.03889745  0.00393978  0.01157793 -0.00114821 -0.02348773
 -0.00016072  0.00388779  0.00102451  0.00436666]
Mean Squared Error on training data: 0.00046703816622017735
Mean Squared Error on testing data: 0.0003627680848403324
R^2 Score: 0.8644015389613166
